## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import re
import math
from datetime import datetime
import time
sys.dont_write_bytecode = True

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import resize

from pathlib import Path
from typing import List, Set, Dict, Tuple, Optional, Iterable, Mapping, Union, Callable

from pprint import pprint
from ipdb import set_trace as brpt

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torch.linalg import norm as tnorm
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision import datasets, transforms

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import loggers as pl_loggers
# Select Visible GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
[print(x) for x in [
    torch.cuda.is_available(),
    torch.cuda.is_initialized(),
    torch.backends.cuda.is_built(),
    torch.backends.cudnn.version(),
    torch.backends.cudnn.is_available(),
    torch.backends.cudnn.enabled,
    torch.backends.cudnn.benchmark
]];

## Set Path 
1. Add project root and src folders to `sys.path`
2. Set DATA_ROOT to `maptile_v2` folder

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SRC = ROOT/'src'
DATA_ROOT = Path("/data/hayley-old/maptiles_v2/")
paths2add = [this_nb_path, ROOT]

print("Project root: ", str(ROOT))
print('Src folder: ', str(SRC))
print("This nb path: ", str(this_nb_path))


for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"\n{str(p)} added to the path.")
        
# print(sys.path)



In [ ]:
# from src.data.datasets.maptiles import Maptiles, MapStyles
from src.data.datamodules.maptiles_datamodule import MaptilesDataModule

from src.data.transforms.transforms import Identity
from src.models.plmodules.three_fcs import ThreeFCs

from src.visualize.utils import show_timgs

## Start experiment 
Given a maptile, predict its style as one of OSM, CartoVoyager

In [ ]:
# for i in [1,2,3]:
#     for j in [1,2,3]:
#         # Instantiate data module
#         cities = ['paris']
#         styles = ['OSMDefault', 'CartoVoyagerNoLabels']
#         zooms = ['14']
#         dm = MaptilesDataModule(data_root=DATA_ROOT,
#                                 cities=cities,
#                                 styles=styles,
#                                 zooms=zooms)

#         # Instantiate the pl Module
#         nh1, nh2 = 100*i,100*j
#         dim_in = dm.in_size**2*dm.n_channels
#         n_classes = len(dm.styles)
#         model = ThreeFCs( dim_in=dim_in, nh1=nh1, nh2=nh2, n_classes=n_classes)
#         print(model.hparams)
#         # Instantiate a PL `Trainer` object
#         # -- most basic trainer: uses good defaults, eg: auto-tensorboard logging, checkpoints, logs, etc.
#         # -- Pass the data module along with a pl module
#         # ref: https://www.learnopencv.com/tensorboard-with-pytorch-lightning/
#         tb_logger = pl_loggers.TensorBoardLogger(save_dir='lightning_logs', name='three_fcs')
#         trainer_config = {
#             'gpus':1,
#             'max_epochs': 20,
#             'progress_bar_refresh_rate':10,
#             'auto_lr_find': True,
#             'terminate_on_nan':True,
#             'val_check_interval': 0.25, #iterations
#             'logger':tb_logger
#         }
#         trainer = pl.Trainer(**trainer_config)
#         # trainer = pl.Trainer(fast_dev_run=True)

#         trainer.fit(model, dm)

#         # Finally,
#         # Log this model's hyperparmeters to tensorboard
#         hparams = dict(model.hparams)
#         metrics = {'hparam/acc': model.acc.compute().item()}
#         model.logger.experiment.add_hparams(hparam_dict=hparams,
#                                             metric_dict=metrics) #how to store the 'best' value of the metric?
#         # Alternatively, use pl.Logger's method "log_hyperparameters"
# #         logger.log_hyperparams(hparams, metrics)

In [ ]:
# # Log this model's hyperparmeters to tensorboard
# model.logger.experiment.add_hparams(hparam_dict=dict(model.hparams), 
#                                     metric_dict={'hparam/acc': model.acc.compute().item()}) #how to store the 'best' value of the metric?

In [ ]:
from src.models.plmodules.vanilla_vae import VanillaVAE
from pl_bolts.callbacks import LatentDimInterpolator


In [ ]:
# Instantiate data module
cities = ['paris']
styles = ['OSMDefault', 'CartoVoyagerNoLabels']
zooms = ['14']
dm = MaptilesDataModule(data_root=DATA_ROOT,
                        cities=cities,
                        styles=styles,
                        zooms=zooms,
                       bs=1)

# Instantiate the pl Module
in_shape = (3,64,64)
latent_dim = 10
hidden_dims = [32,64,128,256,512]
act_fn = nn.LeakyReLU()
model = VanillaVAE(in_shape, 
                     latent_dim,
                     hidden_dims,
                     act_fn)
print(model.hparams)

In [ ]:
# Instantiate a PL `Trainer` object
# -- most basic trainer: uses good defaults, eg: auto-tensorboard logging, checkpoints, logs, etc.
# -- Pass the data module along with a pl module
# ref: https://www.learnopencv.com/tensorboard-with-pytorch-lightning/
tb_logger = pl_loggers.TensorBoardLogger(save_dir='lightning_logs', name='vanilla_vae')
trainer_config = {
    'gpus':1,
    'max_epochs': 200,
    'progress_bar_refresh_rate':20,
    'auto_lr_find': True,
    'terminate_on_nan':True,
    'val_check_interval': 0.25, #iterations
    'logger':tb_logger,
    'callbacks':[LatentDimInterpolator()]
}
trainer = pl.Trainer(**trainer_config)
# trainer = pl.Trainer(fast_dev_run=True)

trainer.fit(model, dm)

# Finally,
# Log this model's hyperparmeters to tensorboard
# hparams = dict(model.hparams)
# metrics = {'hparam/acc': model.hparams["loss"]}
# model.logger.experiment.add_hparams(hparam_dict=hparams,
#                                     metric_dict=metrics) #how to store the 'best' value of the metric?
# Alternatively, use pl.Logger's method "log_hyperparameters"
#         logger.log_hyperparams(hparams, metrics)

In [ ]:
#2
# Instantiate data module
cities = ['paris']
styles = ['OSMDefault', 'CartoVoyagerNoLabels']
zooms = ['14']
dm = MaptilesDataModule(data_root=DATA_ROOT,
                        cities=cities,
                        styles=styles,
                        zooms=zooms,
                       bs=1)

# Instantiate the pl Module
in_shape = (3,64,64)
latent_dim = 10
hidden_dims = [32,64,128,256,512]
act_fn = nn.LeakyReLU()
model = VanillaVAE64(in_shape, 
                     latent_dim,
                     hidden_dims,
                     act_fn)
print(model.hparams)
# Instantiate a PL `Trainer` object
# -- most basic trainer: uses good defaults, eg: auto-tensorboard logging, checkpoints, logs, etc.
# -- Pass the data module along with a pl module
# ref: https://www.learnopencv.com/tensorboard-with-pytorch-lightning/
tb_logger = pl_loggers.TensorBoardLogger(save_dir='lightning_logs', name='vanilla_vae')
trainer_config = {
#     'gpus':1,
    'max_epochs': 200,
    'progress_bar_refresh_rate':20,
    'auto_lr_find': True,
    'terminate_on_nan':True,
    'val_check_interval': 0.25, #iterations
    'logger':tb_logger
}
trainer = pl.Trainer(**trainer_config)
# trainer = pl.Trainer(fast_dev_run=True)

trainer.fit(model, dm)

# Finally,
# Log this model's hyperparmeters to tensorboard
# hparams = dict(model.hparams)
# metrics = {'hparam/acc': model.hparams["loss"]}
# model.logger.experiment.add_hparams(hparam_dict=hparams,
#                                     metric_dict=metrics) #how to store the 'best' value of the metric?
# Alternatively, use pl.Logger's method "log_hyperparameters"
#         logger.log_hyperparams(hparams, metrics)

In [ ]:
#3
# Instantiate data module
cities = ['paris']
styles = ['OSMDefault', 'CartoVoyagerNoLabels']
zooms = ['14']
dm = MaptilesDataModule(data_root=DATA_ROOT,
                        cities=cities,
                        styles=styles,
                        zooms=zooms,
                       bs=1)

# Instantiate the pl Module
in_shape = (3,64,64)
latent_dim = 20
hidden_dims = [32,64,128,256,512]
act_fn = nn.LeakyReLU()
model = VanillaVAE64(in_shape, 
                     latent_dim,
                     hidden_dims,
                     act_fn)
print(model.hparams)
# Instantiate a PL `Trainer` object
# -- most basic trainer: uses good defaults, eg: auto-tensorboard logging, checkpoints, logs, etc.
# -- Pass the data module along with a pl module
# ref: https://www.learnopencv.com/tensorboard-with-pytorch-lightning/
tb_logger = pl_loggers.TensorBoardLogger(save_dir='lightning_logs', name='vanilla_vae')
trainer_config = {
#     'gpus':1,
    'max_epochs': 200,
    'progress_bar_refresh_rate':20,
    'auto_lr_find': True,
    'terminate_on_nan':True,
    'val_check_interval': 0.25, #iterations
    'logger':tb_logger
}
trainer = pl.Trainer(**trainer_config)
# trainer = pl.Trainer(fast_dev_run=True)

trainer.fit(model, dm)

# Finally,
# Log this model's hyperparmeters to tensorboard
# hparams = dict(model.hparams)
# metrics = {'hparam/acc': model.hparams["loss"]}
# model.logger.experiment.add_hparams(hparam_dict=hparams,
#                                     metric_dict=metrics) #how to store the 'best' value of the metric?
# Alternatively, use pl.Logger's method "log_hyperparameters"
#         logger.log_hyperparams(hparams, metrics)


In [ ]:
# 4
# Instantiate data module
cities = ['paris']
styles = ['OSMDefault', 'CartoVoyagerNoLabels']
zooms = ['14']
dm = MaptilesDataModule(data_root=DATA_ROOT,
                        cities=cities,
                        styles=styles,
                        zooms=zooms,
                       bs=1)

# Instantiate the pl Module
in_shape = (3,64,64)
latent_dim = 30
hidden_dims = [32,64,128,256,512]
act_fn = nn.LeakyReLU()
model = VanillaVAE64(in_shape, 
                     latent_dim,
                     hidden_dims,
                     act_fn)
print(model.hparams)
# Instantiate a PL `Trainer` object
# -- most basic trainer: uses good defaults, eg: auto-tensorboard logging, checkpoints, logs, etc.
# -- Pass the data module along with a pl module
# ref: https://www.learnopencv.com/tensorboard-with-pytorch-lightning/
tb_logger = pl_loggers.TensorBoardLogger(save_dir='lightning_logs', name='vanilla_vae')
trainer_config = {
#     'gpus':1,
    'max_epochs': 200,
    'progress_bar_refresh_rate':20,
    'auto_lr_find': True,
    'terminate_on_nan':True,
    'val_check_interval': 0.25, #iterations
    'logger':tb_logger
}
trainer = pl.Trainer(**trainer_config)
# trainer = pl.Trainer(fast_dev_run=True)

trainer.fit(model, dm)

# Finally,
# Log this model's hyperparmeters to tensorboard
# hparams = dict(model.hparams)
# metrics = {'hparam/acc': model.hparams["loss"]}
# model.logger.experiment.add_hparams(hparam_dict=hparams,
#                                     metric_dict=metrics) #how to store the 'best' value of the metric?
# Alternatively, use pl.Logger's method "log_hyperparameters"
#         logger.log_hyperparams(hparams, metrics)

## pl.Metrics Module
python-lightning provides a class of metrics that inherits from `nn.Module`
`Metrics` base class's `forward(x)` method does the 2 following actions:
- Calls `update()` on its input `x`
- Simultaneously, returns the value of the metric over the input

Other key methods:
- `Metric.update()`
- `Metric.compute()`
- `Metric.reset()`


    

In [ ]:
pl.metrics.

In [ ]:
from src.visualize.utils import show_timgs, show_timg

In [ ]:
n_samples = 36
with torch.no_grad():
    sampled_recons = model.samaple(n_samples, model.device)
    show_timgs(sampled_recons.detach())

In [ ]:
# recons
with torch.no_grad():
    for n in range(n_samples):
        x,y = next(iter(dm.train_dataloader()))
        mu, log_var,recon = model(x)["mu"], model(x)["log_var"], model(x)["recon"]
        show_timg(recon.detach().squeeze())
        plt.show()
